In [1]:
#genetic algoritm (binary)
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
class GeneticAlgoritmBinary:
    def __init__(self, population_size, parrents_in_population,  num_gens, backpack_lift, epochs, chooser='rulet'):
        """
        n_gens: gen quantity in chromosom
        first_n_population: 
        """
        self.population_size = population_size
        self.parrents_in_population = parrents_in_population
        self.n_gens = num_gens
        self.backpack_lift = backpack_lift
        self.epochs = epochs
        self.items = self.generate_items()
        self.chooser_selector = chooser
        
        
    def generate_items(self):
        d = {'czapka': {'weight': 2, 'price': 3},
             'kurtka': {'weight': 3, 'price': 6},
             'skarpety':{'weight': 1, 'price': 2},
             'buty':{'weight': 4, 'price': 3},
             'woda':{'weight': 2, 'price': 10},
             'kanapki':{'weight': 3, 'price': 7},
             'owoce':{'weight': 4, 'price': 5},
             'kawa':{'weight': 2, 'price': 2},
             'mlotek':{'weight': 6, 'price':1},
             'kilof':{'weight': 7, 'price': 0},
             'kartofle': {'weight': 5, 'price': 4},
             'czekolada': {'weight': 2, 'price': 9},
             'noz': {'weight': 1, 'price': 7},
             'humor': {'weight': 0, 'price': 15},
            }
        return d
    
    def chooser_container(self):
        _chooser = {'rulet': self.rulet_chooser,
                     'ranking': self.ranking_chooser,
                    'tournament': self.tournament_chooser,
        }
        return _chooser

    def create_chromosoms(self):
        arr = np.random.rand(self.population_size, self.n_gens)
        population = np.where(arr < .5, 0, 1)
        population = population.astype(float)
        return population
    
    def is_to_heavy(self, arr, items, backpack_lift):        
        items_keys = list(items.keys())
        s = self.map_arr(items=items, arr=arr, weight='weight').sum()
        if s > backpack_lift:
            return True
        
    def get_chomosom_value(self, arr, items):
        s = self.map_arr(items=items, arr=arr, price='price').sum()        
        return s
    
    def get_weights(self, arr, items):
        s = self.map_arr(items=items, arr=arr, weight='weight').sum()        
        return s    
    
    def check_number_items_in_backpack(self):
        """
        Check if number in container are similar than gen amount in chromosom
        """
        items = len(self.generate_items())
        if items != self.n_gens:
            raise ValueError(f'Items numbers != n_gens. {len(self.items)}|{self.n_gens}.Try add stuff to backpack.')
        else:
            return True

    def map_arr(self, items, arr, **kwargs):
        """
        arr is chromosom
        **kwargs is "weights" or "price" becouse we have in container as value eg.
        {'weight': 7, 'price': 0}
        """
#         print(arr, arr.shape)
        self.check_number_items_in_backpack()
        element = list(kwargs.values())[0]        
        new_arr = []
        items_keys = list(items.keys())
        for i, ele in enumerate(arr):
            value = items[items_keys[i]][element]# can be weight or value
            if ele != 0:
                new_arr.append(value)
            else:
                new_arr.append(ele)
        return np.array(new_arr)       
    
    
    def create_population(self, chromosom_population):
        items = self.generate_items()            
        chromosoms_values = []
        weights = []
        for chromos in chromosom_population:
            value = self.get_chomosom_value(arr=chromos, items=items)
            weight = self.get_weights(arr=chromos, items=items)
            chromosoms_values.append(value)
            weights.append(weight)    
        population = np.insert(arr=chromosom_population, obj=chromosom_population.shape[1], 
                        values=weights, axis=1)        
        population = np.insert(arr=population, obj=population.shape[1], 
                        values=chromosoms_values, axis=1)     
        return population
    
    def set_values(self, chromosom_population, max_lift):       
        weights = chromosom_population[:,-2]        
        value = chromosom_population[:,-1]
        mask = np.where(weights <= max_lift, False, True)        
        values = np.where(mask == False, value, 0) 
        chromosom_population[:,-1] = values
        return chromosom_population
    
    def generate_population(self, chromosom_population):
        population = self.create_population(chromosom_population=chromosom_population)        
        weights, values = population[-2:]            
        population = self.set_values(chromosom_population=population, 
                                     max_lift=self.backpack_lift)
        return population
        
    def add_adataption(self, population):
        """
        population: chromosoms with weight and price
        """
        summary_values = population[:,-1].sum()
        adaptation = population[:,-1] / summary_values        
        population = np.insert(arr=population, 
                               obj=population.shape[1],
                               values=adaptation,
                               axis=1)        
        return population
    
    
    def rulet_chooser(self, population, n_parrents):        
        """
        rulet chooser
        n_parrents - how many parrents we want in set.
                 
        """        
        adaptation = population[:,-1]     
        choosen_parrents_ind = np.random.choice(a=range(0,population.shape[0]), size=n_parrents, p=adaptation)        
        parrents = population[choosen_parrents_ind]        
        return parrents
    
    
    def ranking_chooser(self, population, n_parrents):
        """
        ranking chooser
        """
        mask = population[:,-1].argsort()
        sorted_population = population[mask]
        return sorted_population[::-1][:n_parrents, :]
    
    def tournament_chooser(self, population, n_parrents):        
        chromosoms_in_group = 100
        best_in_groups = []
#         breakpoint()
        for _ in range(n_parrents):            
            # choose id-s chromosoms
            choosen_ind = np.random.choice(len(population), chromosoms_in_group)
            group = population[choosen_ind]
            best_in_group = np.argmax(group[:,-1])            
            best_in_groups.append(group[best_in_group])            
        best_in_groups = np.array(best_in_groups)        
        return best_in_groups        
    
        
    
    def add_mutation(self, child):
        mutation_ind = np.random.choice(a=len(child), size=1)
        child[mutation_ind] = 0 if child[mutation_ind] == 1 else 1
        return child
    
    def create_child(self, split_ind, arr1, arr2, add_mutation=True):
        chromosomA = arr1
        chromosomB = arr2        
        ch1A, ch1B  = chromosomA[:split_ind], chromosomA[split_ind:]
        ch2A, ch2B = chromosomB[:split_ind], chromosomB[split_ind:]        
        childA = np.append(ch1A, ch2B)
        childB = np.append(ch2A, ch1B)
        if add_mutation:
            childA = self.add_mutation(child=childA)
            childB = self.add_mutation(child=childB)        
        return childA, childB
    
    def create_childs(self, population):
        """
        popultion with values, weight, adaptation
        return: pure chromosoms
        """
        childs = []
        spl_ind = self.n_gens//2
        for i in range(0, population.shape[0], 2):
            parrentA = population[i][:-3]
            parrentB = population[i+1][:-3]
            childA, childB = self.create_child(split_ind=spl_ind,
                                               arr1=parrentA,
                                               arr2=parrentB,
                                               add_mutation=True)            
            l = [childA, childB]            
            for child in l:
                price = self.map_arr(items=self.items,
                                arr=child, 
                                price='price').sum()         
                weight = self.map_arr(items=self.items,
                                arr=child, 
                                weight='weight').sum()                
                child = np.append(child,(weight, price))       
                childs.append(child)
        childs = np.array(childs, dtype=float)
        childs = self.set_values(chromosom_population=childs, 
                        max_lift=self.backpack_lift)
#         set default zeros to adaptation
        zeros = np.zeros(childs.shape[0])
        childs = np.insert(childs, childs.shape[1], zeros, axis=1)
        return childs
        
    def drop_duplicates(self, population):
        popul = [tuple(chromosom) for chromosom in population[:,:-3]]        
        unique, ind = np.unique(popul, return_index=True, axis=0)       
        return population[ind]
    
    def merge_childs_and_parrents(self, parrents, childs):
        merged = np.insert(parrents, len(parrents), childs, axis=0)
        np.random.shuffle(merged)
        return merged
        
    
    def pipeline(self, global_population, parrents_in_population, **kwargs):
        _chooser_cointener = self.chooser_container()
        selection_chooser = kwargs['chooser']
        selection_chooser = _chooser_cointener[selection_chooser]        
        pattents_with_adaptation = self.add_adataption(population=population)       
        parrents = selection_chooser(population=pattents_with_adaptation,
                            n_parrents=parrents_in_population)       
        return parrents  
        
    def to_df(self, population):
        df = pd.DataFrame(population)
        weights, values, adaptation = df.columns[-3:]
        df = df.rename(columns={values:'values', 
                                weights:'weights',
                               adaptation: 'adaptation'})
        return df
        
    
    def main(self, verbose=False):        
        global_poplulation =self.generate_population(self.create_chromosoms())        
        global_best_price = 0
        best_global_chromosom = []
        progress_price = []
        for gen in range(self.epochs):
            global_population = self.drop_duplicates(population=global_poplulation)
            global_population = self.pipeline(global_population=global_population,
                                      parrents_in_population=self.parrents_in_population, 
                                      chooser=self.chooser_selector)
#             breakpoint()
            ind_best_local_price = np.argmax(global_population[:,-2])
            best_local_price =  global_population[ind_best_local_price][-2]
            if global_best_price < best_local_price:
                global_best_price = best_local_price
                best_global_chromosom =  global_population[ind_best_local_price]
            best_chromosom =  best_global_chromosom[:-3]
            weight, value, adaptation = best_global_chromosom[-3:]
            if verbose:
                print(f'gen -> {gen + 1}')
                print(f"best chromosm:{best_chromosom.astype(int)}\nweight: {int(weight)}\nprice :{int(value)}\nadaptation: {adaptation}")
                print('')
            progress_price.append(global_best_price)    
            childs = self.create_childs(population=global_population)
            global_population = self.merge_childs_and_parrents(parrents=global_population,
                                                            childs=childs)   
            
        return best_global_chromosom, global_best_price, np.array(progress_price)
        
    def show_pie_char(self, data):
        """
        visualize best parrents using pie char
        """
        fig1, ax1 = plt.subplots(figsize=(20,20))
        ax1.pie(data[:,-1], labels=range(0,data.shape[0]), autopct='%1.1f%%',
        shadow=True, startangle=90)
        
    def show_progress(self):
        _,_, progress= self.main()
#         print(range(progress.shape[0]), progress)
        plt.figure(figsize=(10,10))
        progress = progress.astype(int)
        plt.plot(range(progress.shape[0]), progress, label='siema')
        plt.xlabel('gen')
        plt.ylabel('progress')
        plt.title(f'PROGRESS PLOT :\n{self.chooser_selector}')        
        plt.grid(True)
        plt.show()
        

In [82]:
class GeneticAlgoDecimal(GeneticAlgoritmBinary):
    def __init__(self, population_size, parrents_in_population,  num_gens, backpack_lift, epochs, chooser='rulet'):
        
        super().__init__(population_size, 
                         parrents_in_population,  
                         num_gens, 
                         backpack_lift, 
                         epochs, 
                         chooser='rulet')
        self.bag = self.generate_items()
        
        
    def generate_items(self):
        d = {'czapka': {'weight': 2, 'price': 3, 'amount': 2},
             'kurtka': {'weight': 3, 'price': 6, 'amount': 1},
             'skarpety':{'weight': 1, 'price': 5,'amount': 3},
             'buty':{'weight': 4, 'price': 3, 'amount': 1},
             'woda':{'weight': 2, 'price': 10, 'amount': 5},
             'kanapki':{'weight': 3, 'price': 7, 'amount': 4},
             'owoce':{'weight': 4, 'price': 5, 'amount': 2},
             'kawa':{'weight': 2, 'price': 2, 'amount': 1},
             'mlotek':{'weight': 6, 'price':1, 'amount': 1},
             'kilof':{'weight': 7, 'price': 0, 'amount': 1},
             'kartofle': {'weight': 5, 'price': 4, 'amount': 1},
             'czekolada': {'weight': 2, 'price': 9, 'amount': 3},
             'noz': {'weight': 1, 'price': 7, 'amount': 2},
             'humor': {'weight': 0, 'price': 15, 'amount': 2},
            }
        items = list(d.keys())[:self.n_gens]
        d = {key: d[key] for key in items}                     
        return d
    
    def create_chromosoms(self):
        """Create single chromosom as arr n x 1
        """        
        max_weight_in_bag = max(val['amount'] for item, val in self.bag.items())
        chromosom = np.random.choice(max_weight_in_bag, self.n_gens)        
        return chromosom
    
    def create_population(self):
        """
        Create population containing chromosomes.
        """
        population = []
        for _ in range(self.population_size):
            chromosom = self.create_chromosoms()
            population.append(chromosom)
        return np.array(population)
    
    
    def map_allels(self, chromosom, **kwargs):
        """
        kwargs: 'element' might be weight, price.  
        **kwargs is "weights" or "price" becouse we have in container as value eg.
        {'weight': 7, 'price': 0}
        """
        bag = self.bag
        self.check_number_items_in_backpack()        
        element = kwargs['element']        
        maped_allels_to_values = []
        bag_keys = list(bag.keys())
        # allel is value of gen in chromosom
        for i, allel in enumerate(chromosom):            
            # items_keys[i] item in bag
            value = bag[bag_keys[i]][element]# can be weight or value            
            # multiplayed quantity allel by value (weight, price according on type of element)
            multiplayed = allel * value
            maped_allels_to_values.append(multiplayed)          
        return np.array(maped_allels_to_values)
    
    def calc_quantity(self, population):
        """
        Compare bag items quantity and allel value (allel is responsible for quantity items in chromosom)
        return: array nx1 size,  where n is size of population
            True if all allels in chromosoms are < than qunatity items in bag otherwise False
        
        
        """
        bag_items_amt = np.array([val['amount'] for key , val in self.bag.items()])
        print(bag_items_amt)
        print()
        mask = bag_items_amt >= population    
        data = np.apply_along_axis(all, axis=1, arr=mask)
        return data.reshape(-1,1)
    
    
    def mesure_adaptation_population(self, chromosom):
        weight = self.map_allels(chromosom=chromosom,
                                element='weight')
        price = self.map_allels(chromosom=chromosom,
                                element='price')
        quantity = ...
    
    def chromosom_pipeline(self):
        """
        Extend chromosom population. Add to chromosoms two new params weight and price
        """
        ...
        
        
        
        
        
    
    

In [50]:
# test line


array([False, False])

TODO
- create method to support createing population chromosoms + weight + price + adaptation
- set all support function to return array as vector nx1
- funkcje pomiaru przystosowania osobnika dostosowa¢ obecnych danych (mesure adaptation population method set to new data)

In [69]:
#hyperparams
epochs = 5
population_size = 10**4
gen_size = 10
max_lift = 40
np.random.seed(42)
parrents_in_population = 200
chooser = 'tournament' 
# inst = GeneticAlgoritmBinary(population_size=population_size,
#                        parrents_in_population=parrents_in_population,                       
#                        num_gens=gen_size,
#                       backpack_lift=max_lift,
#                       epochs=epochs,
#                       chooser=chooser)
# len(inst.items)

# chromosoms = inst.create_chromosoms()
# population = inst.generate_population(chromosom_population=chromosoms)
# population = inst.add_adataption(population=population)
# population = inst.drop_duplicates(population=population)
# d = inst.tournament_chooser(population, parrents_in_population)


# s = inst.ranking_adaptation(population=population)
# items = inst.items
# inst.main(verbose=True)
# inst.show_progress()
# s = s[::-1]
# s[:, :-1]
decimal_inst = GeneticAlgoDecimal(population_size=population_size,
                       parrents_in_population=parrents_in_population,                       
                       num_gens=gen_size,
                      backpack_lift=max_lift,
                      epochs=epochs,
                      chooser=chooser)

d = decimal_inst.create_population()


In [83]:
display(decimal_inst.bag)
mask = decimal_inst.calc_quantity(population=d)
mask.shape

{'czapka': {'weight': 2, 'price': 3, 'amount': 2},
 'kurtka': {'weight': 3, 'price': 6, 'amount': 1},
 'skarpety': {'weight': 1, 'price': 5, 'amount': 3},
 'buty': {'weight': 4, 'price': 3, 'amount': 1},
 'woda': {'weight': 2, 'price': 10, 'amount': 5},
 'kanapki': {'weight': 3, 'price': 7, 'amount': 4},
 'owoce': {'weight': 4, 'price': 5, 'amount': 2},
 'kawa': {'weight': 2, 'price': 2, 'amount': 1},
 'mlotek': {'weight': 6, 'price': 1, 'amount': 1},
 'kilof': {'weight': 7, 'price': 0, 'amount': 1}}

[2 1 3 1 5 4 2 1 1 1]



(10000, 10)

In [81]:
bag_amt = np.array([1,2,3,4])
chrom_amt = np.array([[1,2,1,5],[3,1,2,0], [0,0,0,0]])
mask = bag_amt >= chrom_amt
mask
dd = np.apply_along_axis(all, axis=1, arr=mask).reshape(-1,1)
dd

array([[False],
       [False],
       [ True]])

In [66]:
d

array([[3, 4, 2, ..., 2, 2, 4],
       [3, 2, 4, ..., 4, 0, 3],
       [1, 4, 3, ..., 1, 3, 3],
       ...,
       [2, 3, 2, ..., 1, 4, 1],
       [0, 4, 3, ..., 1, 3, 4],
       [1, 1, 1, ..., 4, 3, 0]])

{0.00014869808071331822: (1, 60),
 0.0001514016821808331: (3, 8),
 0.00015680888511586285: (1, 147),
 0.00015951248658337773: (2, 28),
 0.0001622160880508926: (5, 58),
 0.00016491968951840748: (14, 9),
 0.00016762329098592236: (15, 32),
 0.00017032689245343724: (19, 3),
 0.00017303049392095211: (34, 0),
 0.00017573409538846696: (18, 4),
 0.00017843769685598184: (14, 6),
 0.00018114129832349672: (17, 24),
 0.0001838448997910116: (15, 12),
 0.00018654850125852647: (8, 45),
 0.00018925210272604135: (8, 5),
 0.00019195570419355622: (15, 2),
 0.0001946593056610711: (10, 14),
 0.00019736290712858598: (1, 96)}

In [107]:
 #quick tests
class TestCases():
    
    def test_heavy(self):
        items = inst.generate_items()
        arr = np.array([0,1,1,1,0])
        max_lift = 6
        from_func = inst.is_to_heavy(arr=arr, 
                                     items=items, 
                                     backpack_lift=max_lift)
        expected = True
        assert from_func == expected, f'{from_func} != {expected}'
    
    def test_create_child(self):
        arr1 = np.array([1]*13)
        arr2 = np.array([0]*13)
        spl_ind = 5 
        ch1, ch2 = inst.create_child(split_ind=spl_ind, 
                                 arr1=arr1, 
                                 arr2=arr2, 
                                 add_mutation=False)
        ch1_expect = [1]*5 + [0]*5
        ch2_expect = ch1_expect[::-1]
        ch1m, ch2m = inst.create_child(split_ind=spl_ind, 
                                 arr1=arr1, 
                                 arr2=arr2, 
                                 add_mutation=True)
        assert ch1.tolist() == ch1_expect, f'{ch1.tolist()} != {ch1_expect}'
        assert ch2.tolist() == ch2_expect, f'{ch2.tolist()} != {ch2_expect}'      
        
tests = TestCases()
tests.test_create_child()

[1 1 1 1 1 1 1 1 1 1]
[0 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1] [0 0 0 0 0] [1 1 1 1 1]
[1 1 1 1 1 0 0 0 0 0]
[0 0 0 0 0 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1]
[0 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1] [0 0 0 0 0] [1 1 1 1 1]
[1 1 1 1 1 0 0 0 1 0]
[0 0 0 0 0 1 1 0 1 1]


In [16]:
np.random.seed(42)
a = np.random.uniform(0,20, (3,3)).astype(int)
display(a)
m = np.argmax(a[:,0])
a[m]

array([[ 7, 19, 14],
       [11,  3,  3],
       [ 1, 17, 12]])

array([11,  3,  3])

In [8]:
sum([val['weight'] for key, val in items.items()])

42

In [17]:
aa = np.array([[4,5,6], [1,2,3]])

aa

aa[:,2].argsort()

array([1, 0], dtype=int32)

In [67]:
l1 = range(1,10)
l2 = range(10,20)
list(zip(l1, l2))

[(1, 10),
 (2, 11),
 (3, 12),
 (4, 13),
 (5, 14),
 (6, 15),
 (7, 16),
 (8, 17),
 (9, 18)]

In [11]:
def fo(**kargs):
    val = kargs['price']
    return val
fo(price='kupa')

'kupa'